In [1]:
import wn
from collections import defaultdict
import pandas as pd

# Load lexicon
lexicon = wn.Wordnet('oewn:2024')

# Lấy 2 synset cần thống kê
synset_ids = ['oewn-00021445-n', 'oewn-07571428-n']
synsets = [lexicon.synset(sid) for sid in synset_ids]

def collect_hyponyms_recursive(synset, level=0, visited_synsets=None, all_data=None):
    """
    Thu thập tất cả hyponyms theo đệ quy và thống kê theo level
    visited_synsets: set để tránh trùng lặp synset
    all_data: dict để lưu thống kê theo level {level: {'synsets': set(), 'lemmas': list()}}
    """
    if visited_synsets is None:
        visited_synsets = set()
    if all_data is None:
        all_data = defaultdict(lambda: {'synsets': set(), 'lemmas': []})
    
    # Kiểm tra trùng lặp synset
    synset_id = synset.id
    if synset_id in visited_synsets:
        return all_data
    visited_synsets.add(synset_id)
    
    # Thêm synset vào level hiện tại
    all_data[level]['synsets'].add(synset_id)
    
    # Thêm tất cả lemmas vào level hiện tại (không loại bỏ trùng)
    for lemma in synset.lemmas():
        all_data[level]['lemmas'].append(lemma)
    
    # Duyệt các hyponym
    hyponyms = synset.hyponyms()
    for hyponym in hyponyms:
        collect_hyponyms_recursive(hyponym, level + 1, visited_synsets, all_data)
    
    return all_data

# Thống kê cho từng synset riêng
results_per_synset = {}
for i, synset in enumerate(synsets):
    synset_id = synset_ids[i]
    visited = set()
    data = defaultdict(lambda: {'synsets': set(), 'lemmas': []})
    
    # Thu thập dữ liệu
    collect_hyponyms_recursive(synset, level=0, visited_synsets=visited, all_data=data)
    
    # Chuyển đổi sang dict dễ đọc
    stats = {}
    for level in sorted(data.keys()):
        stats[level] = {
            'synsets_count': len(data[level]['synsets']),
            'lemmas_count': len(data[level]['lemmas']),
            'synsets': sorted(data[level]['synsets']),
            'lemmas': data[level]['lemmas']
        }
    
    # Lưu thông tin đầy đủ của synset gốc
    examples = synset.examples()
    results_per_synset[synset_id] = {
        'stats': stats,
        'total_synsets': len(visited),
        'total_lemmas': sum(len(data[level]['lemmas']) for level in data.keys()),
        'synset_info': {
            'id': synset.id,
            'pos': synset.pos,
            'lemmas': ', '.join(lemma for lemma in synset.lemmas()),
            'definition': synset.definition(),
            'examples': ', '.join(examples) if examples else '(không có)',
            'num_lemmas': len(list(synset.lemmas())),
            'num_direct_hyponyms': len(synset.hyponyms())
        }
    }

# Thống kê chung cho cả 2 synset - đơn giản là tổng của 2 sheet riêng rẽ
# Tìm tất cả các level có trong cả 2 synset
all_levels = set()
for result in results_per_synset.values():
    all_levels.update(result['stats'].keys())

# Tính tổng cho từng level
stats_combined = {}
for level in sorted(all_levels):
    synsets_count = 0
    lemmas_count = 0
    synsets_list = []
    lemmas_list = []
    
    for result in results_per_synset.values():
        if level in result['stats']:
            synsets_count += result['stats'][level]['synsets_count']
            lemmas_count += result['stats'][level]['lemmas_count']
            synsets_list.extend(result['stats'][level]['synsets'])
            lemmas_list.extend(result['stats'][level]['lemmas'])
    
    stats_combined[level] = {
        'synsets_count': synsets_count,
        'lemmas_count': lemmas_count,
        'synsets': sorted(synsets_list),
        'lemmas': lemmas_list
    }

total_combined = {
    'total_synsets': sum(result['total_synsets'] for result in results_per_synset.values()),
    'total_lemmas': sum(result['total_lemmas'] for result in results_per_synset.values())
}

# In kết quả
print("=" * 80)
print("THỐNG KÊ THEO TỪNG SYNSET RIÊNG RẼ")
print("=" * 80)

for synset_id, result in results_per_synset.items():
    info = result['synset_info']
    print(f"\n📊 SYNSET: {info['lemmas']} ({synset_id})")
    print("-" * 80)
    print("THÔNG TIN SYNSET GỐC:")
    print(f"  ID: {info['id']}")
    print(f"  POS: {info['pos']}")
    print(f"  Lemmas: {info['lemmas']}")
    print(f"  Definition: {info['definition']}")
    print(f"  Examples: {info['examples']}")
    print(f"  Số lemmas: {info['num_lemmas']}")
    print(f"  Số hyponyms trực tiếp: {info['num_direct_hyponyms']}")
    print("\nTHỐNG KÊ:")
    print(f"  Tổng số synsets (không trùng): {result['total_synsets']}")
    print(f"  Tổng số lemmas (có trùng): {result['total_lemmas']}")
    print("\nThống kê theo level:")
    print(f"{'Level':<10} {'Số synsets':<15} {'Số lemmas':<15}")
    print("-" * 40)
    for level in sorted(result['stats'].keys()):
        stats = result['stats'][level]
        print(f"{level:<10} {stats['synsets_count']:<15} {stats['lemmas_count']:<15}")

print("\n" + "=" * 80)
print("THỐNG KÊ CHUNG CHO CẢ 2 SYNSET")
print("=" * 80)
print("\nTHÔNG TIN SYNSET GỐC:")
for i, synset in enumerate(synsets):
    synset_id = synset_ids[i]
    info = results_per_synset[synset_id]['synset_info']
    print(f"\n  📌 SYNSET GỐC {i+1}: {synset_id}")
    print(f"    ID: {info['id']}")
    print(f"    POS: {info['pos']}")
    print(f"    Lemmas: {info['lemmas']}")
    print(f"    Definition: {info['definition']}")
    print(f"    Examples: {info['examples']}")
    print(f"    Số lemmas: {info['num_lemmas']}")
    print(f"    Số hyponyms trực tiếp: {info['num_direct_hyponyms']}")

print("\nTHỐNG KÊ:")
print(f"  Tổng số synsets (không trùng): {total_combined['total_synsets']}")
print(f"  Tổng số lemmas (có trùng): {total_combined['total_lemmas']}")
print("\nThống kê theo level:")
print(f"{'Level':<10} {'Số synsets':<15} {'Số lemmas':<15}")
print("-" * 40)
for level in sorted(stats_combined.keys()):
    stats = stats_combined[level]
    print(f"{level:<10} {stats['synsets_count']:<15} {stats['lemmas_count']:<15}")

print("\n" + "=" * 80)
print("CHI TIẾT THEO LEVEL (CHUNG)")
print("=" * 80)
for level in sorted(stats_combined.keys()):
    print(f"\n📈 Level {level}:")
    print(f"  - Số synsets: {stats_combined[level]['synsets_count']}")
    print(f"  - Số lemmas: {stats_combined[level]['lemmas_count']}")
    if level <= 2:  # Chỉ hiển thị chi tiết cho level 0-2 để không quá dài
        print(f"  - Synsets: {', '.join(stats_combined[level]['synsets'][:10])}{'...' if len(stats_combined[level]['synsets']) > 10 else ''}")

# Xuất ra file Excel
print("\n" + "=" * 80)
print("XUẤT RA FILE EXCEL")
print("=" * 80)

filename = 'statistics.xlsx'
print(f"Tên file: {filename}")

# Tạo Excel writer
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    # Sheet 1: food, nutrient (oewn-00021445-n)
    synset_id_1 = synset_ids[0]
    result_1 = results_per_synset[synset_id_1]
    info_1 = result_1['synset_info']
    sheet_name_1 = f"{info_1['lemmas']} ({synset_id_1})"
    
    # Tạo DataFrame cho sheet 1
    data_1 = []
    for level in sorted(result_1['stats'].keys()):
        stats = result_1['stats'][level]
        data_1.append({
            'Level': level,
            'Số synsets': stats['synsets_count'],
            'Số lemmas': stats['lemmas_count']
        })
    df_1 = pd.DataFrame(data_1)
    df_1.to_excel(writer, sheet_name=sheet_name_1, index=False)
    
    # Sheet 2: food, solid food (oewn-07571428-n)
    synset_id_2 = synset_ids[1]
    result_2 = results_per_synset[synset_id_2]
    info_2 = result_2['synset_info']
    sheet_name_2 = f"{info_2['lemmas']} ({synset_id_2})"
    
    # Tạo DataFrame cho sheet 2
    data_2 = []
    for level in sorted(result_2['stats'].keys()):
        stats = result_2['stats'][level]
        data_2.append({
            'Level': level,
            'Số synsets': stats['synsets_count'],
            'Số lemmas': stats['lemmas_count']
        })
    df_2 = pd.DataFrame(data_2)
    df_2.to_excel(writer, sheet_name=sheet_name_2, index=False)
    
    # Sheet 3: Cả 2 synset
    sheet_name_3 = "Cả 2 synset"
    
    # Tạo DataFrame cho sheet 3
    data_3 = []
    for level in sorted(stats_combined.keys()):
        stats = stats_combined[level]
        data_3.append({
            'Level': level,
            'Số synsets': stats['synsets_count'],
            'Số lemmas': stats['lemmas_count']
        })
    df_3 = pd.DataFrame(data_3)
    df_3.to_excel(writer, sheet_name=sheet_name_3, index=False)

print(f"✅ Đã xuất file Excel: {filename}")
print(f"  - Sheet 1: {sheet_name_1}")
print(f"  - Sheet 2: {sheet_name_2}")
print(f"  - Sheet 3: {sheet_name_3}")


THỐNG KÊ THEO TỪNG SYNSET RIÊNG RẼ

📊 SYNSET: food, nutrient (oewn-00021445-n)
--------------------------------------------------------------------------------
THÔNG TIN SYNSET GỐC:
  ID: oewn-00021445-n
  POS: n
  Lemmas: food, nutrient
  Definition: any substance that can be metabolized by an animal to give energy and build tissue
  Examples: (không có)
  Số lemmas: 2
  Số hyponyms trực tiếp: 15

THỐNG KÊ:
  Tổng số synsets (không trùng): 1603
  Tổng số lemmas (có trùng): 2333

Thống kê theo level:
Level      Số synsets      Số lemmas      
----------------------------------------
0          1               2              
1          15              39             
2          91              143            
3          334             504            
4          473             656            
5          411             598            
6          217             311            
7          55              72             
8          6               8              

📊 SYNSET: food, solid 

/Users/hai.phan/Library/Python/3.9/lib/python/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
